# For Attachments

In [3]:
#Importing Libraries
import pandas as pd
import smtplib
import io
import os
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from PyPDF2 import PdfFileWriter, PdfFileReader
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
from configparser import ConfigParser

parser = ConfigParser()
parser.read(r"C:\Users\jaske\OneDrive\Desktop\email\names.ini")

#Reading Members Data
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTLujUkBEqkfW0ncIO10XSV2KpZYSh0SJ9o8Tro0W5ZZgNWdnaOayohQLicxEZlapGcLwtxY8rQk7EQ/pub?output=xlsx"
maindata = pd.read_excel(url)
data = maindata[maindata["Date of Birth"].notnull()]
data = data.reset_index(drop=True)

sender = parser.get("appreciation","sender_email")
password = parser.get("appreciation","sender_password")
mockcard = parser.get("appreciation","card_pdf")
finalcard = parser.get("appreciation","sender_pdf")
appreciate = parser.get("appreciation","appreciatemessage")

#Attaching attachments
try:
    pdf = finalcard
except Exception as e:
    pass
try:
    image = r""
except Exception as e:
    pass

#Sending Mails
for i in data.index:
    if(data["Date of Birth"][i].strftime('%m/%d') == datetime.today().strftime('%m/%d')):

#Adding name to the PDF Birthday Card
        packet = io.BytesIO()
        can = canvas.Canvas(packet)
        can.setFont("Times-Bold", 21)
        can.setFillColor("#7b2365")
        can.drawString(90, 193, data["Name"][i])
        can.save()

        packet.seek(0)
        new_pdf = PdfFileReader(packet)
        existing_pdf = PdfFileReader(open(mockcard, "rb"))
        output = PdfFileWriter()
        page = existing_pdf.getPage(0)
        page.mergePage(new_pdf.getPage(0))
        output.addPage(page)
        outputStream = open(finalcard, "wb")
        output.write(outputStream)
        outputStream.close()
        
        body = "Hi " + data["Name"][i] + ","
        file = open(appreciate, "r")
        body = body + file.read()
        body = body + ""
        file.close()
            
        message = MIMEMultipart()
        message["From"] = sender
        message["To"] = data["Email"][i]
        message["Subject"] = "We appreciate your support during these tough times " + data["Name"][i]
        message.attach(MIMEText(body, "plain"))
        
        try:
            f = open(pdf, 'rb')
            attach = MIMEApplication(f.read(), _subtype="pdf")
            attach.add_header("Content-Disposition", "attachment", filename = "Thank you.pdf")
            message.attach(attach)
            f.close()
        except Exception as e:
            pass
        try:
            f = open(image, 'rb')
            attach = MIMEApplication(f.read(), _subtype="jpg")
            attach.add_header("Content-Disposition", "attachment", filename = "tarezameen.jpg")
            message.attach(attach)
            f.close()
        except Exception as e:
            pass

        session = smtplib.SMTP("smtp.gmail.com", 587)
        session.starttls()
        session.login(sender, password)
        text = message.as_string()
        session.sendmail(sender, data["Email"][i], text)
        session.quit()

# Without Attachments

In [4]:
#Importing Libraries
import pandas as pd
import smtplib
import io
import os
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from PyPDF2 import PdfFileWriter, PdfFileReader
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
from configparser import ConfigParser

#Configuring Senders/receivers detail
data = pd.read_excel(r"C:\Users\jaske\OneDrive\Desktop\NGO\email\appreciation_donor\data.xlsx")
listdict = dict(zip(data["name"], data["email"]))
#sender, password = "info@tarezameenfoundation.org", "year@2021"
sender, password = "contact@tarezameenfoundation.org", "year@1212"
#sender, password = "hr@tarezameenfoundation.org", "tzf@1234"

#Attaching attachments
file = open(r"C:\Users\jaske\OneDrive\Desktop\NGO\email\appreciation_donor\letter.html", "r")
a = file.read()
file.close()
body = a
#Reading Members Data
for i in data.index:
    
#Sending Mails
    message = MIMEMultipart()
    message["Subject"] = "Greetings from Tare Zameen Foundation"
    message["From"] = sender
    message["To"] = data["email"][i]
    message.attach(MIMEText(body,"html"))
    session = smtplib.SMTP("smtp.gmail.com", 587)
    session.starttls()
    session.login(sender, password)
    text = message.as_string()
    session.sendmail(sender, data["email"][i], text)
    session.quit()
    del message